In [17]:
using Distributions
using Plots

abstract type Component end
abstract type Scatterer end

In [2]:
# data stuff
using DelimitedFiles


mutable struct Data1D
    x::Vector
    y::Vector
    yerr::Union{Vector, Nothing}
    xerr::Union{Vector, Nothing}
    fname::Union{String, Nothing}
    Data1D(x::Vector, y::Vector) = new(x, y, nothing, nothing, nothing)
    Data1D(x::Vector, y::Vector, yerr::Vector) = new(x, y, yerr, nothing)
    Data1D(x::Vector, y::Vector, yerr::Vector, xerr::Vector) = new(x, y, yerr, xerr)
    Data1D(x::Vector, y::Vector; yerr::Vector, xerr::Vector) = new(x, y, yerr, xerr)
    function Data1D(x::Vector, y::Vector;
            yerr::Vector,
            xerr::Vector
        )
        new(x, y, yerr, xerr)
    end
    Data1D(filename::String) = read_data(filename)
end


function read_data(filename::String; delim=nothing)
    if delim === nothing
        arr = readdlm(filename)
    else
        arr = readdlm(filename, delim)
    end
    nrows, ncols = size(arr)

    if ncols == 2
        data = Data1D(arr[:, 1], arr[:, 2], nothing, nothing)
    elseif ncols == 3
        data = Data1D(arr[:, 1], arr[:, 2], arr[:, 3], nothing)
    elseif ncols == 4
        data = Data1D(arr[:, 1], arr[:, 2], arr[:, 3], arr[:, 4])
    end
    data.fname = filename
    return data
end


function refresh!(data::Data1D)
   data_updated = read_data(data.fname)
   data.x = data_updated.x
   data.y = data_updated.y
   data.yerr = data_updated.yerr
   data.xerr = data_updated.xerr
   return nothing
end


function Base.size(data::Data1D)
    return size(data.x)
end

function Base.length(data::Data1D)
    return length(data.x)
end

Plots.plot(data::Data1D) = plot(data.x, data.y, yaxis=:log)
Plots.plot!(data::Data1D) = plot!(data.x, data.y, yaxis=:log)

In [3]:
# parameters

mutable struct Parameter
    value::Real
    vary::Bool
    bounds::Distribution
    name::Union{String, Symbol}
    function Parameter(value::Real;
            vary::Bool=false,
            bounds::Distribution=Uniform(-Inf, Inf),
            name::Union{String, Symbol}=""
        )
        new(value, vary, bounds, name)
    end
end

build_parameter(p::Union{Parameter, Real}) = isa(p, Parameter) ? p : Parameter(p)

build_parameter (generic function with 1 method)

In [4]:
# Structure and reflectometry stuff

struct Structure <: AbstractVector{Component}
    components::Vector{Component}
end
Structure() = Structure([])

Base.size(s::Structure) = size(s.components)
Base.getindex(s::Structure, i::Int) = s.components[i]
Base.IndexStyle(::Type{<:Structure}) = IndexLinear()
function Base.push!(s::Structure, items...)
    for item in items
        push!(s.components, item)
    end
end
function Base.append!(s::Structure, arr::Array{N, 1} where N<:Component)
    for item in arr
        push!(s.components, item)
    end
end
Base.setindex!(s::Structure, c::Component, i::Int) = setindex!(s.components, c, i)


mutable struct SLD <: Scatterer
    re::Parameter
    im::Parameter
    SLD(re::Union{Real, Parameter}) = new(build_parameter(re), Parameter(0.0))
    SLD(re::Union{Real, Parameter}, im::Union{Real, Parameter}) = new(
        build_parameter(re), build_parameter(im)
    )
    SLD(sld::Complex) = new(Parameter(sld.re), Parameter(sld.im))
end
sld(s::SLD) = s.re.value + 1.0im * s.im.value


mutable struct Slab <: Component
    thickness::Parameter
    scatterer::Scatterer
    roughness::Parameter
    vfsolv::Parameter
    function Slab(thickness::Union{Parameter, Real},
        scatterer::Union{Scatterer, Real, Complex, Tuple{Real, Real}},
        roughness::Union{Parameter, Real},
        vfsolv::Union{Parameter, Real})
        if isa(scatterer, Scatterer)
            s = scatterer
        else
            s = SLD(scatterer)
        end
        new(build_parameter(thickness), s, build_parameter(roughness), build_parameter(vfsolv)
    )
    end
end
function slabs(slab::Slab)
    sldv = sld(slab.scatterer)
    return [slab.thickness.value, sldv.re, sldv.im, slab.roughness.value]
end


(|)(a::Component, b::Component) = Structure([a, b])
function |(s::Structure, b::Component)
    push!(s, b)
    s
end
function |(s::Structure, arr::Array{N, 1} where N<:Component)
    append!(s, arr)
    s
end

| (generic function with 3 methods)

In [5]:
data = Data1D("c_PLP0011859_q.txt")

Data1D([0.00806022, 0.00813662, 0.00826375, 0.00837067, 0.00845033, 0.00853083, 0.00861217, 0.00869437, 0.00877743, 0.00886136  …  0.42568, 0.429936, 0.434235, 0.438577, 0.442962, 0.447391, 0.451865, 0.456383, 0.460946, 0.465555], [0.709581, 0.862281, 0.908647, 0.773292, 1.05797, 1.01566, 0.734717, 0.769216, 1.11574, 0.972303  …  4.01737e-7, 3.17279e-7, 5.50631e-7, 5.0851e-7, 6.02593e-7, 4.38454e-7, 3.38757e-7, 4.35846e-7, 3.85579e-7, 3.83415e-7], [0.0850676, 0.11237, 0.0790047, 0.0792728, 0.125959, 0.113295, 0.0611566, 0.0617058, 0.11273, 0.089716  …  1.53814e-7, 1.6913e-7, 1.61142e-7, 1.6499e-7, 1.73835e-7, 1.6535e-7, 1.87639e-7, 1.97826e-7, 1.76143e-7, 1.88454e-7], [0.000331422, 0.000334619, 0.000339939, 0.000344412, 0.000347744, 0.000351111, 0.000354512, 0.000357949, 0.000361421, 0.000364929  …  0.0201807, 0.0204321, 0.0206873, 0.0209465, 0.0212097, 0.0214771, 0.0217487, 0.0220245, 0.0223047, 0.0225894], "c_PLP0011859_q.txt")